In [ ]:
import pandas as pd
df_raw = pd.read_csv(r"../Dataset/Canda_CPI_Jan_2020-Oct_2025.csv")

In [ ]:
#Drop columns
drop_cols = [
    "GEO",
    "DGUID",
    "UOM",
    "UOM_ID",
    "SCALAR_FACTOR",
    "SCALAR_ID",
    "VECTOR",
    "COORDINATE",
    "STATUS",
    "SYMBOL",
    "TERMINATED",
    "DECIMALS"

]
df = df_raw.drop(drop_cols, axis=1).copy()

In [ ]:
#Rename columns
df = df.rename(columns={
    "REF_DATE": "date",
    "Products and product groups": "category_raw",
    "VALUE": "cpi"
})

In [ ]:
#Apply proper dtypes
df["date"] = pd.to_datetime(df["date"])
df["cpi"] = pd.to_numeric(df["cpi"], errors="coerce")

In [ ]:
#Add year and month columns
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month

In [ ]:
#Clean category column
dim_category = (
    df[["category_raw"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_category["category"] = (
    dim_category["category_raw"]
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", " ", regex=True)
)

dim_category.head()

In [ ]:
#Join clean unique list of categories back to original
#dataset on "category_raw"

df = df.merge(dim_category, on="category_raw", how="left")
df.head()

In [ ]:
#Create time-series dataset
cpi_over_time = df[[
    "date",
    "year",
    "month",
    "category",
    "cpi"
]].copy()

cpi_over_time = cpi_over_time.sort_values(["category", "date"])
cpi_over_time.head(), cpi_over_time.shape

In [ ]:
#Save the new time-series csv for analysis
cpi_over_time.to_csv("CPI_over_time.csv", index=False)

In [ ]:
#Create a summary table(category x year, average cpi)
annual = (
    cpi_over_time
    .groupby(["category", "year"], as_index=False)
    .agg(avg_cpi=("cpi","mean"))
)
annual.head()

In [ ]:
#sort values
annual = annual.sort_values(["category", "year"])

#Previous year CPI by category
annual["prev_year_cpi"] = (
    annual
    .groupby("category")["avg_cpi"]
    .shift(1)
)

#YOY inflation rate %
annual["yoy_inflation_rate"] = (
    (annual["avg_cpi"] - annual["prev_year_cpi"])
    / annual["prev_year_cpi"] * 100
)

#Drop first year (2020) as it has NaN values
cpi_summary = annual.dropna(subset=["yoy_inflation_rate"]).copy()

cpi_summary.head(), cpi_summary.shape

In [ ]:
cpi_summary.to_csv("cpi_summary.csv", index=False)